<a href="https://colab.research.google.com/github/leehyunggeunkeun/pytorch-study/blob/master/DNN_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MNIST 데이터셋으로 softmax classfication

In [0]:
#edwith 부스트코스 파이토치 강의를 보면서 필사하고 느낀점 및 방법들을 까먹지 않게 정리함...
#출처 edwith 부스트 코스 pytorch 강의 

In [0]:
#필요한 라이브러리 불러오기

#기본
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#기존 데이터셋 불러오기
import torchvision.datasets as dsets         
import torchvision.transforms as transforms  #다양한 이미지 변환

import random

In [0]:
#gpu 사용하기

device = 'cuda' if torch.cuda.is_available() else 'cpu'
random.seed(111)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [0]:
#hyperparameters

lr = 0.001
training_epochs = 15
batch_size = 100

In [0]:
mnist_train = dsets.MNIST(root='MNIST_data/',train=True,transform=transforms.ToTensor(),download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',train=False,transform=transforms.ToTensor(),download=True)

In [0]:
#데이터셋 불러오기

dataloader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)

In [0]:
#layer를 한번 쌓아보자....

linear1=torch.nn.Linear(784,256,bias=True)
linear2=torch.nn.Linear(256,128,bias=True)
linear3=torch.nn.Linear(128,64,bias=True)
linear4=torch.nn.Linear(64,32,bias=True)    
linear5=torch.nn.Linear(32,10,bias=True) 

relu=torch.nn.ReLU()



In [0]:
#초기화
torch.nn.init.normal_(linear1.weight)
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)
torch.nn.init.normal_(linear4.weight)
torch.nn.init.normal_(linear5.weight) 

Parameter containing:
tensor([[-0.8760, -0.2428, -0.0590,  0.1154,  0.1392, -2.1023, -1.3665,  1.6940,
         -0.3535, -0.4622, -0.9144, -2.4040, -1.0246,  1.2613, -0.8845, -0.5964,
          0.8419, -0.2984, -0.2444, -0.7330,  0.6643,  0.3606,  0.0785, -0.0251,
          0.1463,  0.6727, -0.9334,  0.0913, -0.2854,  1.2619,  0.0364,  1.2454],
        [-0.4429,  0.1839,  1.3681, -0.0368, -0.1383,  1.3645,  0.1795,  0.3320,
         -0.7304,  1.1410,  1.8497,  1.3517, -3.2752,  2.0577,  0.7026,  0.6041,
          0.2571,  1.9346, -0.5456, -1.4699,  0.6691, -3.0588, -1.0069, -0.1531,
         -2.0676,  1.0222,  0.0523, -0.5590,  0.2663,  0.7497, -0.9099,  1.4027],
        [ 0.3852,  2.2101, -1.5015, -0.6829,  0.3430, -0.5899, -0.5412, -1.3859,
          1.5074, -0.9125, -0.4844, -0.5480, -0.4807, -0.5619,  0.5002,  1.0479,
          1.5341, -0.9516,  0.0034,  0.6849,  0.0748, -0.2988, -0.8951,  0.0170,
          0.1203, -0.2802, -1.2814,  1.2865,  0.2399,  0.3876,  1.4302, -0.7666],
   

In [0]:
model = torch.nn.Sequential(linear1, relu,linear2, relu,linear3,relu,linear4,relu,linear5).to(device)

In [0]:
criterion = torch.nn.CrossEntropyLoss().to(device)     
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [0]:
total_batch=len(dataloader)


for epoch in range(training_epochs):
    avg_cost=0

    for x,y in dataloader:
        x=x.view(-1,784).to(device)
        y=y.to(device)

        optimizer.zero_grad()
        hypothesis=model(x)
        cost = criterion(hypothesis, y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')


Epoch: 0001 cost = 1526.032958984
Epoch: 0002 cost = 276.694458008
Epoch: 0003 cost = 154.710083008
Epoch: 0004 cost = 100.505905151
Epoch: 0005 cost = 70.059188843
Epoch: 0006 cost = 51.572326660
Epoch: 0007 cost = 38.485992432
Epoch: 0008 cost = 29.354921341
Epoch: 0009 cost = 22.394456863
Epoch: 0010 cost = 17.481502533
Epoch: 0011 cost = 13.946192741
Epoch: 0012 cost = 10.915907860
Epoch: 0013 cost = 8.931630135
Epoch: 0014 cost = 7.279120922
Epoch: 0015 cost = 6.125812054
Learning finished


In [0]:
# Test the model using test sets
with torch.no_grad():
    
    x_test = mnist_test.data.view(-1, 784).float().to(device)
    y_test = mnist_test.targets.to(device)

    prediction = model(x_test)
    correct_prediction = torch.argmax(prediction, 1) == y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    x_single_data = mnist_test.data[r:r + 1].view(-1, 784).float().to(device)
    y_single_data = mnist_test.targets[r:r + 1].to(device)

    print('Label: ', y_single_data.item())
    single_prediction = model(x_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())


Accuracy: 0.9134999513626099
Label:  9
Prediction:  9


cs231n 강의에서도 딴거 말고 걍 Batch Normalization 하라 했음...
Batch Normalization 에 대해 알아보자